<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/Ensembles/First_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

# Set up wandb

In [2]:
import wandb
# config = dict (
#   architecture = "bert_arabic_english",
#   epochs = 1,
#   batch_size = 4
# )

# run = wandb.init(
#   project="capture-disagreement",
#   notes="test_run",
#   #name = "test_run",
#   config=config,
#   save_code = True,
#   tags = ["bert_arabic_english", "weighted_loss", "balanced_loss", "only_last_layer"],
#   #magic = True
# )

run = wandb.init()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Load data

In [4]:
bert_multilingual_last_layer_dataset = artifact = run.use_artifact('capture_disagreement/capture-disagreement/bert_multilingual_last_layer_dataset:v0', type='dataset')
bert_multilingual_last_layer_dataset_directory = bert_multilingual_last_layer_dataset.download()

wandb: Downloading large artifact bert_multilingual_last_layer_dataset:v0, 7371.81MB. 2608 files... 
wandb:   2608 of 2608 files downloaded.  
Done. 0:2:58.8


In [3]:
# import pandas as pd
# import os
# artifact_dataset = run.use_artifact('capture_disagreement/capture-disagreement/dataset_combined_train:v0', type='dataset')
# artifact_dir = artifact_dataset.download()
# df_all = pd.read_csv(os.path.join(artifact_dir, "dataset_combined_train.csv"))

In [5]:
import torch
import os

class ModelOutputDataset(torch.utils.data.Dataset):
    def __init__(self, output_dir):
        self.output_dir = output_dir
        self.output_paths = [os.path.join(output_dir, path) for path in os.listdir(output_dir)]
        
    def __len__(self):
        return len(self.output_paths)
    
    def __getitem__(self, index):
        output_path = self.output_paths[index]
        data = torch.load(output_path, map_location=device)
        return data

In [6]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
bert_multilingual_last_layer_dataset = ModelOutputDataset(bert_multilingual_last_layer_dataset_directory)

In [8]:
# output_dir = bert_multilingual_last_layer_dataset_directory
# output_paths = [os.path.join(output_dir, path) for path in os.listdir(output_dir)]
# output_path = output_paths[0]
# torch.load(output_path, map_location=device)

In [9]:
# bert_multilingual_last_layer_dataset[0][0].last_hidden_state.shape[-1]

# Dataset

In [47]:
import torch
from tqdm import tqdm_notebook


# Split the dataset into train and test
train_set, test_set = torch.utils.data.random_split(bert_multilingual_last_layer_dataset, [0.8, 0.2])

# Create a list of 10 linear layers
lin_layers = [torch.nn.Linear(bert_multilingual_last_layer_dataset[0]["output"].last_hidden_state.shape[-1], 2) for _ in range(10)]

# Move the linear layers to the designated device
for lin_layer in lin_layers:
    lin_layer = lin_layer.to(device)

# Set up an Adam optimizer for each linear layer
optimizers = [torch.optim.Adam(lin_layer.parameters()) for lin_layer in lin_layers]

# Set up a cross-entropy loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Set up a dataloader for the train set
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)#drop_lat bc i messed up data generation

In [50]:
import matplotlib.pyplot as plt

# Train the linear layers
for i, lin_layer in enumerate(lin_layers):
    # Set the current linear layer to training mode
    lin_layer.train()
    
    # Set the current optimizer
    optimizer = optimizers[i]
    
    # Set up lists to store the losses for each epoch
    epoch_losses = []
    
    # Train the current linear layer for multiple epochs
    for epoch in range(5):
        # Set up a running loss for the epoch
        running_loss = 0.0
        
        # Iterate through the train dataloader
        for batch in tqdm_notebook(train_dataloader):
            # Move the inputs and labels to the designated device
            inputs, hard_labels, soft_labels,  = batch['output'].last_hidden_state.to(device), batch['labels'].to(device), batch['soft_labels']#.to(device)
            #num_annotations, annotations =  batch['num_annotations'].to(device), batch['annotations'].to(device)

            #TODO I messed up the data generation method, every idx in the dataset has actually 4 elements(so batch_size gets multipled by 4)
            #Also some weird rescaling is needed, not sure if that is the best option tbh
            inputs = inputs.view(-1, inputs.shape[-2], inputs.shape[-1])
            hard_labels = hard_labels.view(-1).unsqueeze(-1).expand(-1, outputs.shape[-2])

            optimizer.zero_grad()

            outputs = lin_layer(inputs)


            loss = loss_fn(torch.swapaxes(outputs, -1, -2), hard_labels)


            loss.backward()

            optimizer.step()


            running_loss += loss.item()
        

        epoch_losses.append(running_loss)
        
        # Plot the losses for the epoch
        plt.plot(epoch_losses)
        plt.show()





<ipython-input-50-c04b753c3815>:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(train_dataloader):


  0%|          | 0/130 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
# Set up a dataloader for the test set
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

# Evaluate the linear layers on the test set
for i, lin_layer in enumerate(tqdm_notebook.tqdm_notebook(lin_layers, desc='Evaluating Linear Layers')):
    # Set the current linear layer to evaluation mode

    lin_layer.eval()
    
    # Set up a running loss for the evaluation
    running_loss = 0.0
    
    # Set up a running correct count for the evaluation
    running_correct_count = 0
    
    # Iterate through the test dataloader
    for inputs, labels in test_dataloader:
        # Move the inputs and labels to the designated device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        with torch.no_grad():
            outputs = lin_layer(inputs)
            loss = loss_fn(outputs, labels)
        
        # Update the running loss and correct count
        running_loss += loss.item()
        running_correct_count += torch.sum(torch.argmax(outputs, dim=1) == labels).item()
    
    # Calculate the average loss and accuracy for the evaluation
    avg_loss = running_loss / len(test_dataloader)
    avg_accuracy = running_correct_count / len(test_set)
    
    print(f'Linear Layer {i}: loss = {avg_loss}, accuracy = {avg_accuracy}')

In [ ]:
import evaluate

#TODO something is off here, untrained model gets 0.6 im ce eval

def evaluate_f1_score_huggingface(model, eval_dataloader):
  total = 0
  f1 = 0
  f1_metric = evaluate.load("f1")
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    N = predictions.shape[0]
    total+=1#the metric already gives the average, so no + batch_size
    f1 += f1_metric.compute(predictions=predictions, references=hard_labels, average = 'micro')["f1"]#micro must be used, otherwise smth wrong is returned, TODO look into that (00 00 is undefined)

  return f1/total


#slightly different to the pytorch implementations, not sure why TODO
def evaluate_ce_score(model, eval_dataloader, epsilon = 1e-12):
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)                                      
    N = predictions.shape[0]
    total+=N#maybe should be 1 here as well
    ce += -torch.sum(soft_labels*torch.log(predictions+1e-9))
  return ce/total


def evaluate_bce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.BCELoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]["1"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    N = probabilities.shape[0]
    total +=N
    ce += criterion(probabilities[:,-1], soft_labels)
  return ce/total


def evaluate_ce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.CrossEntropyLoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)                          
    N = probabilities.shape[0]
    total+=N
    ce += criterion(probabilities, soft_labels)
  return ce/total



In [ ]:
r_ce = evaluate_ce_score(model, eval_dataloader)
wandb.log({"ce_held_out_score": r_ce})
r_ce

In [ ]:
r_f1 = evaluate_f1_score_huggingface(model, eval_dataloader)
wandb.log({"f1_held_out_score": r_f1})
r_f1

In [ ]:
run.finish()